In [1]:
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/USA_Housing.csv')
df.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population,Price,Address
0,79545.458574,5.682861,7.009188,4.09,23086.800503,1.059034e+06,"208 Michael Ferry Apt. 674\nLaurabury, NE 3701..."
1,79248.642455,6.002900,6.730821,3.09,40173.072174,1.505891e+06,"188 Johnson Views Suite 079\nLake Kathleen, CA..."
2,61287.067179,5.865890,8.512727,5.13,36882.159400,1.058988e+06,"9127 Elizabeth Stravenue\nDanieltown, WI 06482..."
3,63345.240046,7.188236,5.586729,3.26,34310.242831,1.260617e+06,USS Barnett\nFPO AP 44820
4,59982.197226,5.040555,7.839388,4.23,26354.109472,6.309435e+05,USNS Raymond\nFPO AE 09386


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
 6   Address                       5000 non-null   object 
dtypes: float64(6), object(1)
memory usage: 273.6+ KB


In [6]:
X = df[['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms', 'Avg. Area Number of Bedrooms', 'Area Population']]
y = df[['Price']]

In [7]:
X.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,79545.458574,5.682861,7.009188,4.09,23086.800503
1,79248.642455,6.002900,6.730821,3.09,40173.072174
2,61287.067179,5.865890,8.512727,5.13,36882.159400
3,63345.240046,7.188236,5.586729,3.26,34310.242831
4,59982.197226,5.040555,7.839388,4.23,26354.109472


In [8]:
y.head()

,Price
0,1.059034e+06
1,1.505891e+06
2,1.058988e+06
3,1.260617e+06
4,6.309435e+05


In [10]:
## Splitting Data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=42)

In [11]:
## Lets preprocess the data to prevent the leakage. Also the test data will remain unseen during training
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
num_features = X.columns
preprocessor = ColumnTransformer([("num", StandardScaler(), num_features)])

In [12]:
##fit the model. In this case we will use SGDRegressor
from sklearn.linear_model import SGDRegressor
model = SGDRegressor(random_state=42)

In [13]:
##Train the data within a pipeline.
from sklearn.pipeline import Pipeline
pipeline = Pipeline([("preprocessing", preprocessor), ("model", model)])
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  Index(['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population'],
      dtype='object'))])),
                ('model', SGDRegressor(random_state=42))])

In [18]:
##Evaluate - check metrics, accuracy score etc.
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE:", rmse)
print("R2 Score:", r2_score(y_test, y_pred))



RMSE: 100506.69407918518
R2 Score: 0.9178948704085483


In [22]:
#hyperparameters tuning and utilizing Grid Search CV to find out the best parameter
from sklearn.model_selection import GridSearchCV
param_grid = {"model__alpha":[0.0001, 0.001, 0.01], "model__max_iter":[1000, 2000], "model__learning_rate":["constant", "adaptive", "optimal"], "model__eta0":[0.01, 0.1]}

In [23]:
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring = "neg_root_mean_squared_error", n_jobs = -1)
grid.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         Index(['Avg. Area Income', 'Avg. Area House Age', 'Avg. Area Number of Rooms',
       'Avg. Area Number of Bedrooms', 'Area Population'],
      dtype='object'))])),
                                       ('model',
                                        SGDRegressor(random_state=42))]),
             n_jobs=-1,
             param_grid={'model__alpha': [0.0001, 0.001, 0.01],
                         'model__eta0': [0.01, 0.1],
                         'model__learning_rate': ['constant', 'adaptive',
                                                  'optimal'],
                         'model__max_iter': [1000, 2000]},
             scoring='neg_root_mean_squared_error')

In [24]:
##finding the best parameter after tuning
best_model = grid.best_estimator_


In [26]:
y_pred = best_model.predict(X_test)
print(y_pred)

[1308435.09274669 1236922.75432724 1243260.33823204 1228899.74545814
 1063374.2979388  1543639.98245253 1094832.69941482  833750.191298
  788922.98585819 1469431.48524295  672352.67707215 1606216.88110286
 1004380.00101478 1796103.86320916 1288452.07325383 1087929.82486818
 1422911.45965979 1078160.45139922  802562.35306654  931047.57254154
 1134985.55894322  916626.05960264 1489610.79286412 1284581.81276834
 1581628.21712784 1132719.75875325 1090009.97623401  974797.56197576
  924447.69042306 1740408.96449167 1286305.00673661 1620785.50717949
 1435149.7549768  1234177.58691643 1485029.79015018 1717759.63694608
 1538499.18863096  777763.35937305 1764793.80729865 1176088.14803655
 1553331.06311676  898092.23775747 1370830.274174    845607.72550078
 1201057.08396869 1133373.00425596 1363155.99147988 1449412.74791227
 1574179.00735724 1233723.59632504 1484123.31563133 1295076.68036466
 1222006.6000831   990252.27509246 1693377.93711872 1823085.21095921
 1136409.82817456 1282038.41284809 1

In [27]:
print("RMSE:", rmse)
print("R2 Score:", r2_score(y_test, y_pred))

RMSE: 100506.69407918518
R2 Score: 0.9179946086186115


In [28]:
print("Best Parameters:", grid.best_params_)

Best Parameters: {'model__alpha': 0.001, 'model__eta0': 0.01, 'model__learning_rate': 'adaptive', 'model__max_iter': 1000}
